# Reverse Engineering the UR3e IK parameters

Universal Robots posts the [DH parameters](https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/) of all of their robots online, the hope is with these parameters for our robot, and the joint position read off the modbus, we might be able to reverse-engineering the inverse kinematics and write our own controller for working with a floating coordinate system.

In [60]:
import numpy as np
import json
import os
import time

# np.set_printoptions(formatter={'float': '{: 0.5f}'.format})


In [78]:
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

print("Available files:")
files = []
for file in os.listdir():
    if file.endswith(".json"):
        # print("\t" + file)
        files.append(file)

i = 9

path = os.path.abspath("..\\IK_Scans") + "\\" + files[i]

print(path)

with open(path, 'r') as infile:
    json_data = json.load(infile)


joints_at_points = json_data["joints_at_points"]
start_joints = joints_at_points[0][1]

print(np.rad2deg(start_joints))
names = ("Floating coordinate system", "Joint angles", "Base position")
for i in range(3):
    print(f"{names[i]}: {joints_at_points[0][i]}")
    
# print(joints_at_points[1])

Available files:
c:\Users\ander\OneDrive - UW\Robotics lab material\Robotics Control Code\Ben's Control Code\IK_Scans\Test_08_16__18_45.json
[[ 174.86672]
 [ 337.64403]
 [ 80.38598]
 [ 212.28086]
 [ 270.37878]
 [ 95.68395]]
Floating coordinate system: [[0, 0, 0], [0, 0, 0]]
Joint angles: [[3.052], [5.893], [1.403], [3.705], [4.719], [1.67]]
Base position: [[[0.4321], [0.0936], [-0.2023]], [[-0.289], [-3.121], [-0.002]]]


In [79]:
#               theta,a,d,alpha     (radians and meters)
# Theta represents the current orientation of that particular joint
tcp_offset = 0.08
DH = np.array([[0, 0, 0.1519,np.pi/2],
                [0,-0.2434,0.,0.],
                [0,-0.2132,0.,0.],
                [0,0.,0.1311,np.pi/2],
                [0,0.,0.0854,-np.pi/2],
                [0,0.,0.0921+tcp_offset,0.]])

def matrix_to_coordinates(mat):
    '''Accepts as input a 4x4 numpy array representing a transformation matrix
    and returns that transformation in an np array of the form
    [X,Y,Z, beta, alpha, gamma]'''
    X = mat[0,3]
    Y = mat[1,3]
    Z = mat[2,3]

    beta = np.arctan2((mat[0,0]**2 + mat[1,0]**2)**0.5,-mat[2,0])
    cosb=np.cos(beta)
    alpha = np.arctan2(mat[0,0]/cosb, mat[1,0]/cosb)
    gamma = np.arctan2(mat[2,2]/cosb, mat[2,1]/cosb)
    return np.array([X,Y,Z,beta,alpha,gamma])

def joints_forward_kinematics_messy(joints):
    '''Accepts as input a list of 6 joint positions in radians'''
    joints = np.array(joints).flatten()
    # joints = np.append(joints,0)
    DH_params = DH.copy()
    DH_params[:,0] = joints

    j_pos = []
    for i in range(6):
        theta,a,d,alpha = DH_params[i,:].tolist()
        
        trans_1 = np.eye(4)
        trans_1[2,3] = d

        rot_1 = np.array([[np.cos(theta), -np.sin(theta),0,0],
                        [np.sin(theta), np.cos(theta),0,0],
                        [0,0,1,0],
                        [0,0,0,1]])
        
        trans_2 = np.eye(4)
        trans_2[0,3] = a

        rot_2 = np.array([[1,0,0,0],
                        [0,np.cos(alpha), -np.sin(alpha),0],
                        [0,np.sin(alpha), np.cos(alpha),0],
                        [0,0,0,1]])
        j_pos.append(np.matmul(trans_1,
                        np.matmul(rot_1,
                        np.matmul(trans_2,rot_2))))
    
    t = j_pos[0]
    for i in range(5):
        t = np.matmul(t,j_pos[i+1])

    return matrix_to_coordinates(t)

def joints_forward_kinematics(joints):
    '''Accepts as input a list of 6 joint positions in radians'''
    joints = np.array(joints).flatten()
    # joints = np.append(joints,0)
    DH_params = DH.copy()
    DH_params[:,0] = joints

    j_pos = []
    for i in range(6):
        cos_io = np.cos(DH_params[i,0])
        sin_io = np.sin(DH_params[i,0])
        cos_i3 = np.cos(DH_params[i,3])
        sin_i3 = np.sin(DH_params[i,3])

        j_pos.append(np.array([
            [cos_io, -sin_io * cos_i3, sin_io * sin_i3, DH_params[i,1] * cos_io],
            [sin_io, cos_io * cos_i3, -cos_io * sin_i3, DH_params[i,1] * sin_io],
            [0., sin_i3, cos_i3,DH_params[i,2]],
            [0.,0.,0.,1.]
            ]))
    
    # (T1,T2,T3,T4,T5,T6) = tuple(j_pos)
    # t = T1
    t = j_pos[0]
    for i in range(5):
        t = np.matmul(t,j_pos[i+1])
    

    return matrix_to_coordinates(t)


In [80]:


index = 0
joints = joints_at_points[index][1]
base_pos = joints_at_points[index][2]
base_pos = np.array(base_pos).flatten()
floating_pos = joints_at_points[index][0]

# ("Floating coordinate system", "Joint angles", "Base position")

# print(np.array(start_joints).flatten())
calc = joints_forward_kinematics(joints)
# print(calc)
# print(base_pos)

# error = []
start_position = joints_forward_kinematics(joints_at_points[0][1])

for i in range(20):
    joints = joints_at_points[i][1]
    base_pos = joints_at_points[i][2]
    base_pos = np.array(base_pos).flatten()
    floating_pos = np.array(joints_at_points[i][0]).flatten() * 0.001

    # ("Floating coordinate system", "Joint angles", "Base position")

    calc = joints_forward_kinematics(joints)
    # print(calc)
    # print(base_pos)
    print(f"Starting position: {start_position}")
    print(f"Current position:  {calc}")
    print(f"Change:            {calc - start_position}")
    print(f"Normalized translational change, actual: {np.sum((calc-start_position)[0:3]**2)**0.5}")
    print(f"Motion in floating coordinate system: {floating_pos}")
    print(f"Normalized translational change, floating: {np.sum(floating_pos[0:3]**2)**0.5}")
    print("---------------------")
    # print(f"Non-dimensional error {np.sum((calc-base_pos)[0:3]**2)**0.5}")


# print(error)

Starting position: [ 0.43259  0.09391 -0.10894  1.57685  1.75959  1.57684]
Current position:  [ 0.43259  0.09391 -0.10894  1.57685  1.75959  1.57684]
Change:            [ 0.00000  0.00000  0.00000  0.00000  0.00000  0.00000]
Normalized translational change, actual: 0.0
Motion in floating coordinate system: [ 0.00000  0.00000  0.00000  0.00000  0.00000  0.00000]
Normalized translational change, floating: 0.0
---------------------
Starting position: [ 0.43259  0.09391 -0.10894  1.57685  1.75959  1.57684]
Current position:  [ 0.43259  0.09391 -0.10894  1.57685  1.75959  1.57684]
Change:            [ 0.00000  0.00000  0.00000  0.00000  0.00000  0.00000]
Normalized translational change, actual: 0.0
Motion in floating coordinate system: [ 0.00000  0.00000  0.00000  0.00000  0.00000  0.00000]
Normalized translational change, floating: 0.0
---------------------
Starting position: [ 0.43259  0.09391 -0.10894  1.57685  1.75959  1.57684]
Current position:  [ 0.43285  0.09389 -0.10795  1.57586  1.

In [71]:
print(DH)
theta,d,a,alpha = DH[2,:].tolist()
print(d)

[[ 0.00000  0.00000  0.15190  1.57080]
 [ 0.00000 -0.24340  0.00000  0.00000]
 [ 0.00000 -0.21320  0.00000  0.00000]
 [ 0.00000  0.00000  0.13110  1.57080]
 [ 0.00000  0.00000  0.08540 -1.57080]
 [ 0.00000  0.00000  0.22210  0.00000]]
-0.2132
